必读文档:
- [Effective Go](https://go.dev/doc/effective_go)
- [Go Specification](https://go.dev/ref/spec)

其他常用:
- [fmt doc](https://pkg.go.dev/fmt)
- jupyter kernel: [GoNB](https://github.com/janpfeifer/gonb)
    - help line magic: `%help`

## 1. base

- go 的常见的引用类型: `slice`, `map`, `chan`, `interface`, `func`, 传递的是 header, 其指向实际的数据.
- [*zeroed value*](https://go.dev/ref/spec#The_zero_value): `nil`
- A slice is a descriptor of an array segment, structure: 
```go
type slice struct {
	array unsafe.Pointer
	len   int
	cap   int
}
```

## 2. create

- [make](https://go.dev/doc/effective_go#allocation_make): creates slices, maps, and channels only, and it returns an initialized (not zeroed) value of type T (not *T). *make* initializes the internal data structure.
- [new](https://go.dev/doc/effective_go#allocation_new): returns a pointer to a newly allocated, zeroed slice structure(a pointer to a nil slice value)

一种另类的创建: *type conversions*: `s := []int(nil)`

In [1]:
%%

// 1. make
var slice_make []int = make([]int, 3)
slice_make[1] = 1
fmt.Printf("slice_make=%+v, len=%d, cap=%d, is nil=%v\n", slice_make, len(slice_make), cap(slice_make), slice_make == nil)

// 2. new
var slice_point_new *[]int = new([]int)
// slice_point_new: a pointer to a nil slice value. 不能直接修改:
// (*slice_point_new)[0] = 1 // panic: runtime error: index out of range [0] with length 0
fmt.Printf("slice_point_new=%+v, len=%d, cap=%d, is nil=%v, point to nil=%v\n", slice_point_new, len(*slice_point_new), cap(*slice_point_new), slice_point_new == nil, *slice_point_new == nil)

// 3. type conversions
var slice_type_conv = []int(nil)
fmt.Printf("slice_type_conv=%+v, is nil: %+v", slice_type_conv, slice_type_conv == nil)

slice_make=[0 1 0], len=3, cap=3, is nil=false
slice_point_new=&[], len=0, cap=0, is nil=false, point to nil=true
slice_type_conv=[], is nil: true

### make

- `make` 初始化 `slice` 时至少要传 *Type, len*, 可额外传第三个参数作为 *cap*, 不传时会默认 *cap=len*
- `make` 的 *cap* 不能小于 *size*

In [40]:
%%

// 指定 len, 默认 cap=len
slice_with_size := make([]int, 5)
fmt.Printf("slice_with_size=%+v, len=%d, cap=%d\n", slice_with_size, len(slice_with_size), cap(slice_with_size))

// 指定 len 与 cap
slice_with_size_cap := make([]int, 5, 8)
fmt.Printf("slice_with_size_cap=%+v, len=%d, cap=%d\n", slice_with_size_cap, len(slice_with_size_cap), cap(slice_with_size_cap))

// len > cap, 如 make([]int, 5, 1), 会报编译错误: invalid argument: length and capacity swapped

slice_with_size=[0 0 0 0 0], len=5, cap=5
slice_with_size_cap=[0 0 0 0 0], len=5, cap=8


## 3. iter

`for [... :=] range`: array, slice, string, or map, or reading from a channel. 

In [30]:
%%

nums := []int{10, 20, 30}
fmt.Printf("shapes: %+v\n", nums)

for idx := range nums {
    fmt.Printf("idx: %d, value: %+v\n", idx, nums[idx])
}

for idx, value := range nums {
    fmt.Printf("idx: %d, value: %+v\n", idx, value)
}

shapes: [10 20 30]
idx: 0, value: 10
idx: 1, value: 20
idx: 2, value: 30
idx: 0, value: 10
idx: 1, value: 20
idx: 2, value: 30


### 关于 for 迭代

推荐阅读: [golang-for-range](https://draveness.me/golang/docs/part2-foundation/ch05-keyword/golang-for-range/)
此处直接引用其结论:
- 所有的 range 循环，Go 语言都会在编译期将原切片或者数组赋值给一个新变量( copy )并获取切片的长度. 所以在循环中追加新的元素也不会改变循环执行的次数
- 迭代元素时，Go 语言会额外创建一个新的变量存储切片/数组中的元素，会在每一次迭代被重新赋值为当前迭代元素。总之: 索引和元素始终都是同一内存空间，直接修改不会影响原 slice。修改的方式: 要么对 `slice[idx]`重新赋值，要么原 slice 的元素是指针时修改解引用的数据

In [60]:
%%

nums := []int{10, 20}

// 1. 迭代时追加
for idx, value := range nums {
    nums = append(nums, idx + 50)
    fmt.Printf("idx: %d, value: %+v\n", idx, value)
}

// 2. 索引和元素始终同一内存，元素是重新赋值而来
for idx, value := range nums {
    fmt.Printf("idx: %d, idx addr: %v, value: %+v, value addr: %v\n", idx, &idx, value, &value)
}

idx: 0, value: 10
idx: 1, value: 20
idx: 0, idx addr: 0xc00008a018, value: 10, value addr: 0xc00008a030
idx: 1, idx addr: 0xc00008a018, value: 20, value addr: 0xc00008a030
idx: 2, idx addr: 0xc00008a018, value: 50, value addr: 0xc00008a030
idx: 3, idx addr: 0xc00008a018, value: 51, value addr: 0xc00008a030


In [97]:
func printSlice(label string, s []*int){
    fmt.Printf("points %s: [", label)
    for idx, iter_value := range s {
        if idx != 0 {
            fmt.Print(" ")
        }
        fmt.Printf("%v", *iter_value)
    }
    fmt.Println("]")
}

In [98]:
%%

nums := []int{10, 20}

// 2.1 由于元素是重新赋值而来所以修改数据时不会影响原数据, struct 重新赋值会 copy
fmt.Printf("nums: %v\n", nums)
for _, value := range nums {
    value = value + 50
}
fmt.Printf("nums after for-range modify range's value: %v\n", nums)

// 2.2 修改 指针slice 的元素会影响原slice
a, b := 10, 20
num_points := []*int {&a, &b}
printSlice("", num_points)
for _, iter_value := range num_points {
    fmt.Printf("iter_value: %v, iter_value's addr: %v, iter_value's val: %v\n", iter_value, &iter_value, *iter_value)
    *iter_value = *iter_value + 50
}
printSlice("after for-range modify range's value", num_points)

nums: [10 20]
nums after for-range modify range's value: [10 20]
points : [10 20]
iter_value: 0xc00008e058, iter_value's addr: 0xc000090020, iter_value's val: 10
iter_value: 0xc00008e060, iter_value's addr: 0xc000090020, iter_value's val: 20
points after for-range modify range's value: [60 70]


## 4. append -- TODO

append 会返回


In [100]:
// todo

### 4.1 growslice

以下均基于 go 1.21.5 版本

[runtime.slice#growslice](https://github.com/golang/go/blob/go1.21.5/src/runtime/slice.go)

扩容:
1. 新长度大于原容量的两倍则新容量直接等于新长度
2. 如果原容量小于 *threshold*(=256) 就会将容量翻倍
3. 使用扩容公式: `newcap += (newcap + 3*threshold) / 4` 直到新容量大于等于新长度

内存对齐:


In [111]:
%%

// newLen > doublecap
nums := []int{1, 2}
fmt.Printf("len: %d, cap: %d, nums: %v\n", len(nums), cap(nums), nums)

numsAd2 := append(nums, 3, 4)
fmt.Printf("len: %d, cap: %d, nums: %v\n", len(numsAd2), cap(numsAd2), numsAd2)

numsAd3 := append(nums, 3, 4, 5)
fmt.Printf("len: %d, cap: %d, nums: %v\n", len(numsAd3), cap(numsAd3), numsAd3)

numsAd4 := append(nums, 3, 4, 5, 6)
fmt.Printf("len: %d, cap: %d, nums: %v\n", len(numsAd4), cap(numsAd4), numsAd4)

len: 2, cap: 2, nums: [1 2]
len: 4, cap: 4, nums: [1 2 3 4]
len: 5, cap: 6, nums: [1 2 3 4 5]
len: 5, cap: 6, nums: [1 2 3 4 5]


### 4.2 slice 的切片

`slice[start:end]` 是引用